In [ ]:
import numpy as np
import torch
import torchvision
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import torch.nn.init
import torch.optim as optim
from torch.autograd import Variable
from torch.nn.functional import batch_norm
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
!pip install wandb
import wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.7 MB/s eta 0:00:00


In [ ]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
Aborted!


In [ ]:
entity_name="team_zulkar"

project_name="CS6910_Assignment_2A"

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
def unaug_data():
    data = '/content/gdrive/My Drive/inaturalist_12K'

    ## Unaugmented data


    data_transforms = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])


    #train dataset
    train_dataset = datasets.ImageFolder(root=f"{data}/train", transform=data_transforms)
    #test dataset
    test_dataset = datasets.ImageFolder(root=f"{data}/val", transform=data_transforms)

    initial_size = len(train_dataset)
    train_size = int(0.8 * initial_size)
    val_size = initial_size - train_size

    # Spliting the train dataset into train and validation data
    train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

    # Loading the dataset
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True, num_workers=2)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=True, num_workers=2)

    return  train_loader, test_loader, val_loader

In [ ]:
def aug_data():
    data = '/content/gdrive/My Drive/inaturalist_12K'

    ## augmented data


    train_transform = transforms.Compose([transforms.RandomHorizontalFlip(p=0.5),
              transforms.RandomVerticalFlip(p=0.5),
              transforms.RandomRotation((120)),
              transforms.RandomApply(torch.nn.ModuleList([transforms.ColorJitter()]), p=0.5),
              transforms.Resize((224,224)),
              transforms.ToTensor(),
              transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    test_transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])


    #train dataset
    train_dataset = datasets.ImageFolder(root=f"{data}/train", transform=train_transform)
    #test dataset
    test_dataset = datasets.ImageFolder(root=f"{data}/val", transform=test_transform)

    initial_size = len(train_dataset)
    train_size = int(0.8 * initial_size)
    val_size = initial_size - train_size

    # Spliting the train dataset into train and validation data
    train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

    # Loading the dataset
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True, num_workers=2)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=True, num_workers=2)
    return  train_loader, test_loader, val_loader

In [ ]:
print(torch.cuda.is_available())

True


In [ ]:
def Resnet_50():
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  model= models.resnet50(pretrained=True).to(device)

  for param in model.parameters():
    param.requires_grad = False

  num_inputs = model.fc.in_features
  model.fc = nn.Sequential(
               nn.Linear(num_inputs, 2048),
               nn.ReLU(inplace=True),
               nn.Dropout(0.3),
               nn.Linear(2048, 10))
  return model

In [ ]:
def resnet_training(data_augmentation):

  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  model=Resnet_50().to(device)
  optimizer = optim.Adam(model.parameters(),lr=1e-3)
  criterion = nn.CrossEntropyLoss().to(device)

  if data_augmentation:
        train_loader, test_loader, val_loader = aug_data()
  else:
        train_loader, test_loader, val_loader = unaug_data()

  num_epochs=25
  gpu_train = torch.cuda.is_available()
  wandb.init()

  for epoch in tqdm(range(1, num_epochs+1)):
      train_loss = 0.0
      val_loss = 0.0
      val_accuracy = 0.0
      train_accuracy=0.0
      total=0.0
      correct=0.0


      model.train()
      for batch in train_loader:
              inputs, labels = batch
              if gpu_train:
                inputs, labels = inputs.cuda(), labels.cuda()
              optimizer.zero_grad()
              outputs = model(inputs)
              loss = criterion(outputs, labels)
              loss.backward()
              optimizer.step()
              train_loss += loss.item()*inputs.size(0)
              _, predicted = torch.max(outputs, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      train_accuracy = 100 * correct / total
      print(f"Train Accuracy: {train_accuracy:.4f}%")


      model.eval()
      for batch in val_loader:
            inputs, labels = batch
            if gpu_train:
              inputs, labels = inputs.cuda(), labels.cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()*inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

      val_accuracy = 100 * correct / total
      print(f"Validation Accuracy: {val_accuracy:.4f}%")


      train_loss = train_loss/len(train_loader.dataset)
      val_loss = val_loss/len(val_loader.dataset)
      val_accuracy = val_accuracy
      train_accuracy = train_accuracy

      wandb_log = True
      if(wandb_log==True):
        epoc=0
        log_dict = {"Train_loss": train_loss, "Validation_loss": val_loss, "Validation_Accuracy": val_accuracy,"Train_Accuracy": train_accuracy}
        epoc=epoc+1
        print('Epoch: {} \tTraining Loss: {:.4f} \tValidation Loss: {:.4f} \tValidation Accuracy: {:.4f} \Train Accuracy: {:.4f}'.format(epoc,
          train_loss, val_loss,val_accuracy, train_accuracy))

      wandb.log(log_dict)

  wandb.run.save()
  wandb.run.finish()
  return model



In [ ]:
trained_model=resnet_training(False)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


  0%|          | 0/25 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Train Accuracy: 60.8272%


  4%|▍         | 1/25 [18:29<7:23:50, 1109.60s/it]

Validation Accuracy: 62.9448%
Epoch: 1 	Training Loss: 1.1999 	Validation Loss: 0.8667 	Validation Accuracy: 62.9448 \Train Accuracy: 60.8272
Train Accuracy: 69.5489%


...

 96%|█████████▌| 24/25 [1:10:18<02:16, 136.33s/it]

Validation Accuracy: 81.7773%
Epoch: 1 	Training Loss: 0.4707 	Validation Loss: 0.8920 	Validation Accuracy: 81.7773 \Train Accuracy: 83.7686
Train Accuracy: 84.7057%


100%|██████████| 25/25 [1:12:34<00:00, 174.18s/it]
wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Validation Accuracy: 83.1068%
Epoch: 1 	Training Loss: 0.4497 	Validation Loss: 0.7880 	Validation Accuracy: 83.1068 \Train Accuracy: 84.7057


Train_Accuracy,▁▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇███
Train_loss,█▅▅▄▄▄▄▃▃▃▃▃▂▃▂▂▂▂▂▂▂▁▁▁▁
Validation_Accuracy,▁▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇███
Validation_loss,▇▆▇▆▅▃▂▃▅▂▄▁▃▂▄▅▃▂▂▃▆▃▅█▄
Train_Accuracy,84.70574
Train_loss,0.44965
Validation_Accuracy,83.10676
Validation_loss,0.78796
